In [110]:
from __future__ import absolute_import
from __future__ import print_function
import scipy.io
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from six.moves import range
import keras.utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re



auto_coded_df = pd.read_csv('../input/balanced_trainers.csv')
y = np.array([y for y in auto_coded_df['Y']])
auto_texts = list(auto_coded_df['Tweet'])

texts = auto_texts

MAX_NUM_WORDS = 100
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
GLOVE_DIR = '../../wqvectors/glove.6B/'

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
MAX_SEQUENCE_LENGTH = len(max(sequences, key=len)) + 100
#MAX_SEQUENCE_LENGTH = 1000

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = y
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
shuffle_split = False
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

import os
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input, Dense

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
h = GlobalMaxPooling1D()(embedded_sequences)

output = Dense(1, activation='sigmoid')(h)

model = Model(inputs=sequence_input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])
hist = model.fit(x_train, y_train, epochs=30, batch_size=100, validation_data=(x_val, y_val))
#model.save('../model_objects/topic_classifier.h5')  # creates a HDF5 file 'my_model.h5'



Found 5715 unique tokens.
Shape of data tensor: (2000, 125)
Shape of label tensor: (2000,)
Found 400000 word vectors.
Train on 1600 samples, validate on 400 samples
Epoch 1/30
1600/1600 [==============================] - 0s 194us/step - loss: 0.5565 - accuracy: 0.7469 - val_loss: 0.4534 - val_accuracy: 0.8525
Epoch 2/30
1600/1600 [==============================] - 0s 98us/step - loss: 0.3996 - accuracy: 0.8531 - val_loss: 0.3572 - val_accuracy: 0.8700
Epoch 3/30
1600/1600 [==============================] - 0s 102us/step - loss: 0.3428 - accuracy: 0.8606 - val_loss: 0.3350 - val_accuracy: 0.8850
Epoch 4/30
1600/1600 [==============================] - 0s 99us/step - loss: 0.3124 - accuracy: 0.8769 - val_loss: 0.3186 - val_accuracy: 0.8850
Epoch 5/30
1600/1600 [==============================] - 0s 132us/step - loss: 0.2859 - accuracy: 0.8875 - val_loss: 0.3100 - val_accuracy: 0.8900
Epoch 6/30
1600/1600 [==============================] - 0s 100us/step - loss: 0.2664 - accuracy: 0.8975 - v